## Grading Script

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import random

#### Import Testing Data

In [2]:
df = pd.read_csv("Data/loon-flights-2021Q1.csv", nrows=1000000)
df = df.sort_values(['flight_id', 'time'])

In [3]:
df.iloc[0, :]
df = df.ffill()

#### Split Data by Flight ID

In [4]:
flights = df.drop(columns=['time']).groupby('flight_id')
flight_count = len(flights)

train_idx = set(random.sample(range(flight_count), 13))
test_idx = set(range(flight_count)) - train_idx

train_flights = pd.concat([group for i, (key, group) in enumerate(flights) if i in train_idx]).groupby('flight_id')
test_flights = pd.concat([group for i, (key, group) in enumerate(flights) if i in test_idx]).groupby('flight_id')

In [5]:
df.columns

Index(['flight_id', 'time', 'latitude', 'longitude', 'altitude', 'temperature',
       'pressure', 'earth_ir', 'earth_ir_sensor_config', 'acs', 'propeller_on',
       'velocity_u', 'velocity_v', 'omega', 'acceleration', 'solar_elevation',
       'solar_azimuth', 'is_daytime'],
      dtype='object')

#### Import Model

In [6]:
input_size = 16
output_size = 2

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, output_size)

    def forward(self, x):   
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
model = Model()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [7]:
for epoch in range(50):
    epoch_loss = 0
    for flight, data in train_flights:
        _data = data.drop(columns='flight_id')
        lat_lon = _data.loc[:, ['latitude', 'longitude']]
        true_positions = torch.tensor(lat_lon.values, dtype=torch.float32)
        data_tensor = torch.tensor(_data.values, dtype=torch.float32)

        predictions = model(data_tensor[:-1, :])
        loss = F.mse_loss(true_positions[1:, :], predictions)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss
    
    print(f"Epoch {epoch} - Loss = {epoch_loss/len(train_flights)}")

Epoch 0 - Loss = 777045.125
Epoch 1 - Loss = 151416.71875
Epoch 2 - Loss = 18491.904296875
Epoch 3 - Loss = 13750.76953125
Epoch 4 - Loss = 5094.7470703125
Epoch 5 - Loss = 2958.547119140625
Epoch 6 - Loss = 2967.300537109375
Epoch 7 - Loss = 2411.093505859375
Epoch 8 - Loss = 2575.261962890625
Epoch 9 - Loss = 2521.79296875
Epoch 10 - Loss = 2357.1630859375
Epoch 11 - Loss = 2339.5634765625
Epoch 12 - Loss = 2122.441650390625
Epoch 13 - Loss = 1989.267822265625
Epoch 14 - Loss = 1937.43896484375
Epoch 15 - Loss = 1859.199462890625
Epoch 16 - Loss = 1793.85498046875
Epoch 17 - Loss = 1720.62060546875
Epoch 18 - Loss = 1649.6573486328125
Epoch 19 - Loss = 1582.8946533203125
Epoch 20 - Loss = 1515.7537841796875
Epoch 21 - Loss = 1445.1788330078125
Epoch 22 - Loss = 1365.6456298828125
Epoch 23 - Loss = 1268.0950927734375
Epoch 24 - Loss = 1196.6905517578125
Epoch 25 - Loss = 1109.8037109375
Epoch 26 - Loss = 1027.8470458984375
Epoch 27 - Loss = 959.1434326171875
Epoch 28 - Loss = 891.5322

### Test Model

In [10]:
losses = {}
num_timesteps = [1]
for timestep in num_timesteps:
    mse = 0
    print(f"Timestep: {timestep}")
    for flight, data in test_flights:
        _data = data.drop(columns='flight_id')
        lat_lon = _data.loc[:, ['latitude', 'longitude']]
        true_positions = torch.tensor(lat_lon.values, dtype=torch.float32)
        data_tensor = torch.tensor(_data.values, dtype=torch.float32)

        # for i in range(_data.shape[0]-timestep):
        #     print(f"{i}th datapoint", end='\r')
        #     X = data_tensor[i:i+timestep, :], 
        #     y = true_positions[i:i+timestep+1:, :]
        #     latlon = y[0]
        #     for step in range(0,timestep):
        #         x = X[step]
        #         x[:, :2] = latlon
        #         latlon = model(x)
        #     loss = F.mse_loss(latlon, y[-1])
        #     mse += loss
        # mse /= _data.shape[0]-timestep

        X = data_tensor[:-timestep, :]
        y = true_positions[timestep:, :]
        pred_y = model(X)
        loss = F.mse_loss(pred_y, y)
        mse += loss

    losses[timestep] = mse / len(test_flights)

Timestep: 1


In [11]:
print(f"Final Losses")
for timestep, loss in losses.items():
    print(f"Timestep {timestep} - Loss = {loss}")

Final Losses
Timestep 1 - Loss = 100.45550537109375
